# FCA News Analysis

Notebook listo para ejecutar que genera datos sintéticos, construye contextos binarios por bloques, ejecuta Formal Concept Analysis (FCA) con la librería `concepts`, y guarda resultados y retículos.

**Instrucciones**: ejecutar las celdas en orden. Si estás en Colab ejecuta primero la celda de instalación y, si es necesario, instala graphviz a nivel sistema (`!apt-get install -y graphviz`).

In [2]:
# 0. Instalación (ejecutar si hace falta)
!pip install pandas concepts graphviz
# En Colab también puede ser necesario:
# !apt-get install -y graphviz

In [3]:
import os
import random
import pandas as pd
import concepts

OUT_DIR = 'formal_concept_analysis/output'
os.makedirs(OUT_DIR, exist_ok=True)

print('Imports listos. OUT_DIR =', OUT_DIR)

Imports listos. OUT_DIR = formal_concept_analysis/output


## 1) Definición de categorías y variables por bloque

In [4]:
GENERO_PERIODISTA = ['Masculino', 'Femenino', 'Agencia/otros medios', 'Redacción', 'Ns/Nc']
GENERO_PERSONAS_MENCIONADAS = ['No hay', 'Sí, hombre', 'Sí, mujer', 'Sí, mujer y hombre']
TEMA = ['Política', 'Deportiva', 'Economía', 'Educación/cultura', 'Tecnología', 'Social', 'Medioambiente']
CITA_TITULAR = ['No', 'Sí']
LENGUAJE_SEXISTA = ['No', 'Sí', 'Sí, además se observa un salto semántico']
BOOLEAN = ['No', 'Sí']

TIPO_FUENTE = ['Político/a', 'Deportista', 'Economista', 'Ciudadano/a', 'Institucional', 'Experto/a', 'Periodista']
GENERO_FUENTE = ['Hombre', 'Mujer', 'NsNc']

CONTENIDO_VARS = ['cita_textual_titular', 'genero_nombre_propio_titular', 'genero_personas_mencionadas',
                  'nombre_propio_titular', 'personas_mencionadas', 'tema', 'menciona_ia', 'ia_tema_central', 'significado_ia']
LENGUAJE_VARS = ['lenguaje_sexista', 'androcentrismo', 'asimetria', 'masculino_generico', 'sexismo_social']
FUENTES_VARS = ['declaracion_fuente', 'genero_fuente', 'nombre_fuente', 'tipo_fuente']

print('Variables definidas')

Variables definidas


## 2) Generador de datos sintéticos

In [5]:
def generate_synthetic_news(n=60, seed=42):
    random.seed(seed)
    rows = []
    for i in range(n):
        periodista = random.choices(GENERO_PERIODISTA, weights=[0.35,0.35,0.05,0.1,0.15])[0]
        tema = random.choice(TEMA)
        tipo_fuente = random.choice(TIPO_FUENTE)
        genero_fuente = random.choices(GENERO_FUENTE, weights=[0.5,0.4,0.1])[0]

        if periodista == 'Masculino' and tema in ['Política','Deportiva']:
            lenguaje = random.choices(LENGUAJE_SEXISTA, weights=[0.25,0.6,0.15])[0]
        elif periodista == 'Femenino' and tema in ['Social','Educación/cultura']:
            lenguaje = random.choices(LENGUAJE_SEXISTA, weights=[0.8,0.18,0.02])[0]
        else:
            lenguaje = random.choices(LENGUAJE_SEXISTA, weights=[0.65,0.3,0.05])[0]

        row = {
            'id': f'N{i+1}',
            'cita_textual_titular': random.choices(CITA_TITULAR, weights=[0.7,0.3])[0],
            'genero_nombre_propio_titular': random.choice(['No hay', 'Sí, hombre', 'Sí, mujer', 'Sí, mujer y hombre']),
            'genero_personas_mencionadas': random.choice(GENERO_PERSONAS_MENCIONADAS),
            'nombre_propio_titular': random.choice(['Ninguno','Persona A','Persona B','Persona C']),
            'personas_mencionadas': random.choice(['ninguno','varias','una']),
            'tema': tema,
            'menciona_ia': random.choices(['No','Sí'], weights=[0.9,0.1])[0],
            'ia_tema_central': random.choices(['No','Sí'], weights=[0.95,0.05])[0],
            'significado_ia': random.choices(['No','Sí'], weights=[0.97,0.03])[0],
            'lenguaje_sexista': lenguaje,
            'androcentrismo': random.choices(BOOLEAN, weights=[0.85,0.15])[0],
            'asimetria': random.choices(BOOLEAN, weights=[0.9,0.1])[0],
            'masculino_generico': random.choices(BOOLEAN, weights=[0.8,0.2])[0],
            'sexismo_social': random.choices(BOOLEAN, weights=[0.9,0.1])[0],
            'declaracion_fuente': random.choices(['No','Sí'], weights=[0.4,0.6])[0],
            'genero_fuente': genero_fuente,
            'nombre_fuente': random.choice(['Fuente A','Fuente B','Fuente C']),
            'tipo_fuente': tipo_fuente
        }
        rows.append(row)
    df = pd.DataFrame(rows)
    return df

# Generar ejemplo y mostrar
df_example = generate_synthetic_news(n=60, seed=123)
df_example.head()

,id,cita_textual_titular,genero_nombre_propio_titular,genero_personas_mencionadas,nombre_propio_titular,personas_mencionadas,tema,menciona_ia,ia_tema_central,significado_ia,lenguaje_sexista,androcentrismo,asimetria,masculino_generico,sexismo_social,declaracion_fuente,genero_fuente,nombre_fuente,tipo_fuente
0,N1,Sí,No hay,"Sí, mujer y hombre",Persona B,varias,Política,No,No,No,No,No,No,No,No,No,Hombre,Fuente C,Periodista
1,N2,No,"Sí, hombre","Sí, hombre",Ninguno,varias,Política,Sí,No,No,No,No,No,No,No,No,Mujer,Fuente C,Economista
2,N3,Sí,"Sí, mujer y hombre","Sí, mujer y hombre",Persona C,varias,Tecnología,No,No,No,No,No,No,No,No,No,NsNc,Fuente C,Periodista
3,N4,No,"Sí, hombre",No hay,Persona B,ninguno,Política,No,Sí,No,No,No,No,No,No,Sí,Hombre,Fuente C,Experto/a
4,N5,No,"Sí, mujer y hombre","Sí, mujer y hombre",Ninguno,ninguno,Tecnología,No,No,No,No,No,No,No,No,No,NsNc,Fuente A,Ciudadano/a


## 3) Convertir a contexto binario (one-hot) por bloque y combinado

In [6]:
def df_to_binary_context(df, vars_to_include, id_col='id'):
    sub = df[[id_col] + [c for c in vars_to_include if c in df.columns]].copy()
    sub = sub.set_index(id_col)
    bin_df = pd.get_dummies(sub)
    return bin_df

# Crear contextos
bin_contenido = df_to_binary_context(df_example, CONTENIDO_VARS)
bin_lenguaje = df_to_binary_context(df_example, LENGUAJE_VARS)
bin_fuentes = df_to_binary_context(df_example, FUENTES_VARS)
bin_combinado = df_to_binary_context(df_example, list(set(CONTENIDO_VARS + LENGUAJE_VARS + FUENTES_VARS)))

print('Contextos creados:')
print(' - contenido shape:', bin_contenido.shape)
print(' - lenguaje shape:', bin_lenguaje.shape)
print(' - fuentes shape:', bin_fuentes.shape)
print(' - combinado shape:', bin_combinado.shape)

# Guardar CSVs
bin_contenido.to_csv(os.path.join(OUT_DIR, 'contenido_context.csv'))
bin_lenguaje.to_csv(os.path.join(OUT_DIR, 'lenguaje_context.csv'))
bin_fuentes.to_csv(os.path.join(OUT_DIR, 'fuentes_context.csv'))
bin_combinado.to_csv(os.path.join(OUT_DIR, 'combinado_context.csv'))
print('CSV contextos guardados en', OUT_DIR)

Contextos creados:
 - contenido shape: (60, 29)
 - lenguaje shape: (60, 11)
 - fuentes shape: (60, 15)
 - combinado shape: (60, 55)
CSV contextos guardados en formal_concept_analysis/output


## 4) Análisis FCA y guardado de retículos

In [7]:
def analyze_fca_and_save(bin_df, name_prefix, top_n=12, out_dir=OUT_DIR):
    os.makedirs(out_dir, exist_ok=True)
    csv_path = os.path.join(out_dir, f'{name_prefix}_context.csv')
    bin_df.to_csv(csv_path)
    ctx = concepts.Context.fromfile(csv_path, frmat='csv')
    concepts_list = []
    print(f"\n--- Análisis FCA: {name_prefix} (mostrando {top_n} conceptos) ---")
    for i, concept in enumerate(ctx.lattice[:top_n]):
        extent = set(concept.extent)
        intent = set(concept.intent)
        concepts_list.append((extent, intent))
        print(f"\nConcepto {i+1}:")
        print("  Extensión (n objetos):", len(extent), sorted(list(extent))[:10])
        print("  Intensión (atributos):", sorted(list(intent))[:20])
    gv_name = os.path.join(out_dir, f'{name_prefix}_lattice')
    try:
        ctx.lattice.graphviz(filename=gv_name, view=False)
        print(f"\n-> Retículo guardado como: {gv_name}.pdf (o .png según Graphviz)")
    except Exception as e:
        print('No se pudo generar gráfica Graphviz:', e)
    return concepts_list

concepts_contenido = analyze_fca_and_save(bin_contenido, 'contenido', top_n=12)
concepts_lenguaje = analyze_fca_and_save(bin_lenguaje, 'lenguaje', top_n=12)
concepts_fuentes = analyze_fca_and_save(bin_fuentes, 'fuentes', top_n=12)
concepts_combinado = analyze_fca_and_save(bin_combinado, 'combinado', top_n=12)


--- Análisis FCA: contenido (mostrando 12 conceptos) ---

Concepto 1:
  Extensión (n objetos): 0 []
  Intensión (atributos): ['cita_textual_titular_No', 'cita_textual_titular_Sí', 'genero_nombre_propio_titular_No hay', 'genero_nombre_propio_titular_Sí, hombre', 'genero_nombre_propio_titular_Sí, mujer', 'genero_nombre_propio_titular_Sí, mujer y hombre', 'genero_personas_mencionadas_No hay', 'genero_personas_mencionadas_Sí, hombre', 'genero_personas_mencionadas_Sí, mujer', 'genero_personas_mencionadas_Sí, mujer y hombre', 'ia_tema_central_No', 'ia_tema_central_Sí', 'menciona_ia_No', 'menciona_ia_Sí', 'nombre_propio_titular_Ninguno', 'nombre_propio_titular_Persona A', 'nombre_propio_titular_Persona B', 'nombre_propio_titular_Persona C', 'personas_mencionadas_ninguno', 'personas_mencionadas_una']

Concepto 2:
  Extensión (n objetos): 60 ['N1', 'N10', 'N11', 'N12', 'N13', 'N14', 'N15', 'N16', 'N17', 'N18']
  Intensión (atributos): []

-> Retículo guardado como: formal_concept_analysis/outp

## 5) Comparación de intents entre bloques

In [8]:
def intents_from_concepts(concepts_list):
    return set([frozenset(intent) for (_, intent) in concepts_list])


def compare_blocks(concepts_by_block):
    intents_dict = {b: intents_from_concepts(c_list) for b, c_list in concepts_by_block.items()}
    blocks = list(intents_dict.keys())
    shared_all = set.intersection(*intents_dict.values()) if len(blocks)>1 else set()
    print('\n=== Intents compartidos por TODOS los bloques (si hay) ===')
    if shared_all:
        for s in shared_all:
            print(sorted(s)[:10])
    else:
        print('Ninguno.')
    print('\n=== Intents exclusivos por bloque ===')
    for b in blocks:
        others = set.union(*[intents_dict[o] for o in blocks if o != b]) if len(blocks) > 1 else set()
        exclusivos = intents_dict[b] - others
        print(f"\n- {b}: {len(exclusivos)} intents exclusivos (mostrando hasta 5):")
        for i, ex in enumerate(list(exclusivos)[:5]):
            print('   ', sorted(list(ex))[:12])

compare_blocks({'contenido': concepts_contenido, 'lenguaje': concepts_lenguaje, 'fuentes': concepts_fuentes})


=== Intents compartidos por TODOS los bloques (si hay) ===
[]

=== Intents exclusivos por bloque ===

- contenido: 1 intents exclusivos (mostrando hasta 5):
    ['cita_textual_titular_No', 'cita_textual_titular_Sí', 'genero_nombre_propio_titular_No hay', 'genero_nombre_propio_titular_Sí, hombre', 'genero_nombre_propio_titular_Sí, mujer', 'genero_nombre_propio_titular_Sí, mujer y hombre', 'genero_personas_mencionadas_No hay', 'genero_personas_mencionadas_Sí, hombre', 'genero_personas_mencionadas_Sí, mujer', 'genero_personas_mencionadas_Sí, mujer y hombre', 'ia_tema_central_No', 'ia_tema_central_Sí']

- lenguaje: 1 intents exclusivos (mostrando hasta 5):
    ['androcentrismo_No', 'androcentrismo_Sí', 'asimetria_No', 'asimetria_Sí', 'lenguaje_sexista_No', 'lenguaje_sexista_Sí', 'lenguaje_sexista_Sí, además se observa un salto semántico', 'masculino_generico_No', 'masculino_generico_Sí', 'sexismo_social_No', 'sexismo_social_Sí']

- fuentes: 1 intents exclusivos (mostrando hasta 5):
    ['

## 6) Búsqueda rápida: conceptos combinados que contienen 'lenguaje_sexista=Sí'

In [9]:
print('\n=== Conceptos combinados con lenguaje_sexista=Sí ===')
for extent, intent in concepts_combinado:
    if any('lenguaje_sexista' in a and '=Sí' in a for a in intent):
        print(' Extensión:', len(extent), ' Intensión sample:', sorted(list(intent))[:12])


=== Conceptos combinados con lenguaje_sexista=Sí ===


## 7) Guardar dataset sintético completo y finalizar

In [ ]:
df_example.to_csv(os.path.join(OUT_DIR, 'noticias_simuladas_full.csv'), index=False)
print('Dataset sintético guardado en', os.path.join(OUT_DIR, 'noticias_simuladas_full.csv'))